# Welcome to Recitation 


## Case Background
In this lab, we continue the analysis of the casting problem. Recall that, as part of a longer
manufacturing process, a factory needs to cast a large number of rectangular metallic blocks.
The blocks are manufactured using a mold, consisting of the main cavity, a cup through
which the molten metal is poured, and two risers for cooling. The size and shape of the
pouring cup and risers affect how quickly the metal can be poured into the mold, how
quickly it cools, and whether it sets correctly.

## Objective
The factory needs to cast batches of 100 blocks of size 4.5×4.5×7 inches. The
current casting approach is conservative – it takes a long time to pour, but the blocks
always set correctly and are usable. Your goal is to achieve a significant reduction in
average casting time while still ensuring that most blocks are usable.



## Variables characterizing the mold

The following nine mold-variables can be varied:
Riser Height, Riser Diameter, Riser 1 Position, Riser 2 Position, Gate Diameter,
Cup Height, Sprue Height, Sprue Diameter Bottom, Sprue Diameter Top.

## Data

To obtain data on how various mold-variable settings affect pouring and cooling, a batch of 100 is poured with random variations in the nine mold-variables, centered about their baseline values. The data is available in castdata.csv. Use the same file as the previous recitation. As before, the first nine columns are the mold-variables listed above and the 10th column is `BatchTime` and the 11th column is `Feasible`. `Feasible` is 1 if the casting is feasible and 0 otherwise.

## ASSIGNMENT

This lab project will guide you through the analysis. First,we have to import castdata.csv into the notebook.

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
df = pd.read_csv('castdata.csv')
df.head() # view the data

We would like to compare the distributions of the predictors among the two populations, Feasible
= 0 and Feasible = 1. The code for doing so is shown below:

In [0]:
'''
Because pandas won't allow multiple scales for the y-axis,
we must divide and conquer with our boxplots. First execute
the follwing code to view the boxplots for the variables
with the lowest values
''' 
low_vals = ['GateDiam', 'SprueDiamBot', 'SprueDiamTop']

df.boxplot(column=low_vals, by = 'Feasible', layout=(2,2), figsize=(10,10))
# layout=(2,2) configures how the plots will be outputted into the console
# for more info on df.boxplot() head to the link:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.boxplot.html

In [0]:
# now do the same for the variables with medium values
med_vals = ['CupHeight', 'RiserHeight', 'Riser1Pos', 'Riser2Pos']

df.boxplot(column=med_vals, by = 'Feasible', layout=(2,2), figsize=(10,10))

In [0]:
# finally, do the same for the highest values
high_vals = ['SprueHeight', 'RiserDiam']

df.boxplot(column=high_vals, by = 'Feasible', layout=(1,2), figsize=(10,10))

To answer the question below, you will need to know about boxplots. A boxplot uses a box to visualize a collection of observations. In the center of the box is a green line. The height of this line is the median of the observations. The height of the top of the box is 75% quantile (the number such that 75% of the observations are below it). The height of the bottom of the box is the 25% quantile. There are lines coming out of the box (called
“whiskers”) that represent the range of values over which most of the data falls. There may be a few circles above and below the whiskers, which are data that fall outside of this range. These points are called “outliers”.

# Question 1
Which two or three variables appear most related to Feasible? For those variables, are they higher or lower on average when Feasible = 1 rather than 0. Use these observations to make an initial recommendation about the optimal values of the mold-variables in order to maximize the probability of a feasible casting.

Ans: 

## Logistic Regression

We will fit a logistic regression model to estimate the probability that a casting is feasible
using the mold-variables as predictors. Fit an initial logistic regression model, using the
following commands:

In [0]:
# pull out our dependent/response variable
y = df["Feasible"]
# slice the df to obtain the desired independent variables 
# sm.add_constant allows sm.Logit to fit an incercept 
X = sm.add_constant(df.loc[:, :'Riser2Pos'])
# fit the model and return the regression results
model = sm.Logit(y, X).fit()
results = model.summary()
results

The `Logit()` function gives the natural log of the odds that y ("Feasible") equals one of the categories (0 or 1). If we were to use the regular values of y, as opposed to $ln(P(Y = y))$, for the response/dependent variable and tried to fit a line, it wouldn’t be a very good representation of the relationship. If you don't believe us, give linear regression a try and see what you find out!

# AIC

AIC stands for Akaike’s Information Criterion. It estimates the quality of a model, relative to each of other models. The lower AIC score is, the better the model is. Therefore, a model with lowest AIC - in comparison to others, is chosen. Execute the code below to print the AIC value of our current model:

In [0]:
print(f'Current model AIC: {model.aic}')
# for those unfamiliar, this is called a Python f-String, it allows you to embed
# variables into a string using brackets

## Question 2

Interpret the summary output: Which variables appear to be statistically
significant at the 95% level, and how do they affect feasibility? Do the results from the
logistic regression agree or disagree with the results from the boxplots?
What is the value of the AIC for this model?

Ans:

## Minimum AIC

We are now going to find a model with the minimum AIC.

In [0]:
# this function finds returns a model with the minimum AIC for a logistic regression model
def minAIC(X,y):
    variables = X.columns
    model = sm.Logit(y,X[variables]).fit()
    while True:
        print(f'old model aic: {model.aic}')
        maxp = np.max(model.pvalues)
        newvariables = variables[model.pvalues < maxp]
        newmodel = sm.Logit(y,X[newvariables]).fit()
        print(f'new model aic: {newmodel.aic}')
        if newmodel.aic < model.aic:
            model = newmodel
            variables = newvariables
        else:
            break
    return model,variables

#select our features
X = sm.add_constant(df.loc[:, :'Riser2Pos'])
# now call the minAIC function on our independent and response variables
new_model, logit_variables = minAIC(X, y)
new_model = sm.Logit(y, X[logit_variables]).fit()
results = new_model.summary()
results

## Question 3

What has changed compared to the previous model? Has the AIC value decreased (check this below)?

In [0]:
# Your code here to check the new_model's AIC value


Ans:

#Multivariate Regression

We will now fit a multivariate regression model for batch time using the same variables as predictors.

Fit a multivariate regression model, using the code below. 

Note that in the linear model, all mold-variables are used, though GateDiam is replaced by a cubic polynomial in GateDiam; this model was suggested by the analysis in the previous lab. Additionally, we have used orthogonal polynomials to avoid colinearity. This will allow us to make inferences about the effect of input variables on batch time. You may recall seeing an error warning or colinearity while doing last week's recitation. Remember, colinearity is a concern for inference but not for prediction.  

Feasible is not used since it is a response, not a mold-variable.

In [0]:
#function to produce orthogonal polynomials
def ortho_poly_fit(x, degree = 1):
    n = degree + 1
    x = np.asarray(x).flatten()
    if(degree >= len(np.unique(x))):
            stop("'degree' must be less than number of unique points")
    xbar = np.mean(x)
    x = x - xbar
    X = np.fliplr(np.vander(x, n))
    q,r = np.linalg.qr(X)

    z = np.diag(np.diag(r))
    raw = np.dot(q, z)

    norm2 = np.sum(raw**2, axis=0)
    alpha = (np.sum((raw**2)*np.reshape(x,(-1,1)), axis=0)/norm2 + xbar)[:degree]
    Z = raw / np.sqrt(norm2)
    return Z, norm2, alpha
#get degree three orthogonal polynomials for 'GateDiam'
Z, norm2, alpha = ortho_poly_fit(df['GateDiam'], degree=3)
df2 = pd.DataFrame(Z, columns = ['const', 'GateDiamOrtho', 'GateDiamSq', 'GateDiamCub'])

#Construct y and X with all orginal variables, higher order 'GateDiam' terms and a constant term.
df2 = pd.DataFrame(Z, columns = ['const', 'GateDiamOrtho', 'GateDiamSq', 'GateDiamCub'])
X = sm.add_constant(pd.concat([df.loc[:, 'CupHeight':'Riser2Pos'], df2.loc[:,'GateDiamOrtho':]], axis=1, sort=False))
y = df["BatchTime"]

linear_model = sm.OLS(y, X).fit()
linear_model.summary()

Run the code below to the linear model and set of variables with minimal AIC. 

In [0]:
def minAIC_OLS(X,y):
    variables = X.columns
    model = sm.OLS(y,X[variables]).fit()
    while True:
        print(f'old model aic: {model.aic}')
        maxp = np.max(model.pvalues)
        newvariables = variables[model.pvalues < maxp]
        newmodel = sm.OLS(y,X[newvariables]).fit()
        print(f'new model aic: {newmodel.aic}')
        if newmodel.aic < model.aic:
            model = newmodel
            variables = newvariables
        else:
            break
    return model,variables
new_linear_model , linear_variables = minAIC_OLS(X,y)
new_linear_model = sm.OLS(y,X[linear_variables]).fit()
results = new_linear_model.summary()
results

##Question 4
What model for predicting BatchTime is selected by minAIC_OLS? Are all variables in that model statistically significant at 0.05?

Ans:

Now we will use the logistic model for `Feasible` and the linear model for `BatchTime`, to inform how to set the values of the mold-variables to minimize expected `BatchTime` and maximize $P(`Feasible = 1)$.

##Question 5
For each of the mold-variables and each of the outcomes $P(Feasible =1)$ and `BatchTime`, state whether increasing the variable would increase the outcome, decrease it, or have no effect, according to the last statistical models that you fit for each.

Hint: Consider the following plot to when trying to understand the effect of `GateDiam` on `BatchTime`. The plot shows how $\beta_{GateDiamOrtho}* X_{GateDiamOrtho} + \beta_{GateDiamSq}*X_{GateDiamSq}+\beta_{GateDiamCub}* X_{GateDiamCub}$ changes as `GateDiam` increases.  

In [0]:
GateDiamPoly = X[['GateDiamOrtho','GateDiamSq','GateDiamCub']].dot(new_linear_model.params[['GateDiamOrtho','GateDiamSq','GateDiamCub']])
plt.scatter(df['GateDiam'],GateDiamPoly)
plt.xlabel('GateDiam')
plt.ylabel('GateDiamPoly')

Ans:

## Question 6
Our goal is to set the mold-variables to minimize expected `BatchTime` and
maximize $P(Feasible = 1)$. Based on your answers to question 5, for each variable state whether it should be as large as possible, as small as possible, or the value is unknown. For example, if increasing a variable will increase the probability of feasibility and decrease batch time, then it should be as large as possible. If it would increase the probability of feasibility but increase batch time, then the value is unknown.

Ans:

# Randomized Search for Better Operating Conditions

We now search to find values of the mold-variables that give a low expected `BatchTime` subject to the probability that the casting is feasible being at least 0.95. This problem is best solved by linear programming, but, since linear programming is not a prerequisite for this course, we will use a randomized search.

Now we create a new set called newdata with 1,000,000 rows where the predictors vary uniformly between their smallest and largest values in the original data. The responses `BatchTime` and `Feasible` are not included in this data set. Rather, the expected values of `BatchTime` and `Feasible` are predicted using the logistic model and the linear model, respectively.

In [0]:
upperbounds = df.loc[:, :'Riser2Pos'].max()
lowerbounds = df.loc[:, :'Riser2Pos'].min()
N = 1000000
np.random.seed(10) 
newdata = pd.DataFrame()
for i in range(9):
  newdata[df.columns[i]] = np.random.uniform(lowerbounds[i],upperbounds[i],N)

newdata = sm.add_constant(newdata)
newdata.head()

## Question 7

We define the rande of our new data set using the lines:
```
upperbounds = df.loc[:, :'Riser2Pos'].max()
lowerbounds = df.loc[:, :'Riser2Pos'].min()
```
Why do we define bounds this way?

Ans:


Next, use `model.predict()` to compute the probability that `Feasible` is equal to 1, and create a sub-dataset called `newdata_feas` of `newdata` where this probability exceeds 0.95. 

Note: be careful when using `.predict()` for classification problems. Depending on the package it will give the probability of the outcome being 1 or a prediction (0 or 1). For example when using sklearn.LogisiticRegression, `.predict()` gives a 0-1 prediction and `.predict_prob()` gives the probability of the outcome being 1. You will explore sklearns's functionality in homework 8. 

In [0]:
#predict probability of feasible block for each data point and remove points 
#with probability less than 0.95
Yhat = new_model.predict(newdata[logit_variables])
newdata_feas = newdata[pd.Series(Yhat >= 0.95)]

Now, predict the value of `BatchTime` for all rows of newdata_feas and find the row where `BatchTime` is minimized; this is the single row of the dataset. 

First we apply the function `ortho_poly_predict` to the `GateDiam` values in our new set so we generate the higher order terms used by the linear regression model. 

In [0]:
def ortho_poly_predict(x, alpha, norm2, degree = 1):
    x = np.asarray(x).flatten()
    n = degree + 1
    Z = np.empty((len(x), n))
    Z[:,0] = 1
    if degree > 0:
        Z[:, 1] = x - alpha[0]
    if degree > 1:
      for i in np.arange(1,degree):
          Z[:, i+1] = (x - alpha[i]) * Z[:, i] - (norm2[i] / norm2[i-1]) * Z[:, i-1]
    Z /= np.sqrt(norm2)
    return Z

Z = ortho_poly_predict(newdata_feas['GateDiam'], alpha, norm2, degree = 3)
Z = pd.DataFrame(Z, columns = ['-', 'GateDiamOrtho', 'GateDiamSq', 'GateDiamCub'])
#Construct X with all orginal variables, higher order 'GateDiam' terms
newdata_feas = newdata_feas.reset_index(drop=True)
X = sm.add_constant(pd.concat([newdata_feas.loc[:, 'CupHeight':'Riser2Pos'], Z.loc[:,'GateDiamOrtho':]], axis=1, sort=False))
#predict batchtime
exBatchTime = new_linear_model.predict(X[variables])

##Question 8 
What is the expected value of BatchTime under optimal conditions? “Optimal
conditions” means that the predictors are set to their optimal values.




Ans:


In [0]:
#your code here

##Question 9
What are the “optimal” values of the variables, where “optimal” means mini-
mizing the expected `BatchTime` subject to $P(Feasible = 1) > 0.95$. Compare these results with your answer to Problem 5. You may also want to look at the maximum and minimum values for each variable. 

Ans:

In [0]:
#your code here

##Question 10 

What is the probability that Feasible will equal 1 under optimal conditions?
Hint: Use predict again.


Ans:

In [0]:
#your code here